In [1]:
import torch
class toyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = torch.nn.Linear(10,10)
        self.dense2 = torch.nn.Linear(10,1)
    
    def forward(self, inputs):
        out = self.dense1(inputs)
        out = self.dense2(out)
        return out

class toyModule2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.toy = toyModule()
    
    def forward(self, inputs):
        return self.toy(inputs)

# Design my tracer

In [2]:
model = toyModule2()

In [11]:
from torch.fx import symbolic_trace as trace3 # cannot be used for transformer models.
from torch.nn import TransformerEncoderLayer, TransformerEncoder

model = TransformerEncoder(TransformerEncoderLayer(d_model=512, nhead=1),num_layers=1)
print(trace3(model).graph)

graph():
graph():
    %src : torch.Tensor [#users=1] = placeholder[target=src]
    %mask : typing.Union[torch.Tensor, NoneType] [#users=1] = placeholder[target=mask](default=None)
    %src_key_padding_mask : typing.Union[torch.Tensor, NoneType] [#users=1] = placeholder[target=src_key_padding_mask](default=None)
    %layers_0 : [#users=1] = call_module[target=layers.0](args = (%src,), kwargs = {src_mask: %mask, src_key_padding_mask: %src_key_padding_mask})
    return layers_0


In [26]:
from torch.fx import Tracer
import math
from typing import Tuple, Callable
from types import CodeType, FunctionType, ModuleType
class MyTracer(Tracer):
    def __init__(self, autowrap_modules: Tuple[ModuleType] = (math, ),
                 autowrap_functions: Tuple[Callable, ...] = (),
                 param_shapes_constant: bool = False) -> None:
        super().__init__(autowrap_modules, autowrap_functions, param_shapes_constant)
        
    def is_leaf_module(self, m: torch.nn.Module, module_qualified_name : str) -> bool:
        print(m.__module__)
        return m.__module__.startswith('torch.nn.modules.linear')

In [28]:
graph = MyTracer().trace(model)

graph():
torch.nn.modules.transformer
torch.nn.modules.activation
torch.nn.modules.dropout
torch.nn.modules.normalization
torch.nn.modules.linear
torch.nn.modules.dropout
torch.nn.modules.linear
torch.nn.modules.dropout
torch.nn.modules.normalization


In [31]:
for node in graph.nodes:
    print(node.)

placeholder
placeholder
placeholder
call_method
call_function
get_attr
get_attr
get_attr
get_attr
call_function
call_function
call_function
call_function
call_function
get_attr
get_attr
call_function
call_module
call_function
call_function
call_module
call_function
call_function
get_attr
get_attr
call_function
output


In [1]:
from transformers import BertConfig, BertModel
from transformers.utils.fx import symbolic_trace

model = BertModel(BertConfig(num_hidden_layers=1))
symbolic_trace(model)

self.orig_fns: {<built-in method full of type object at 0x11d3bf5d0>, <built-in method full_like of type object at 0x11d3bf5d0>, <built-in method eye of type object at 0x11d3bf5d0>, <built-in method zeros of type object at 0x11d3bf5d0>, <built-in method ones of type object at 0x11d3bf5d0>, <built-in method empty of type object at 0x11d3bf5d0>, <built-in method arange of type object at 0x11d3bf5d0>, <built-in method tensor of type object at 0x11d3bf5d0>}
-------------------------------------------------- root:  BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): 

/Users/connolly/opt/anaconda3/envs/python3/lib/python3.7/site-packages/torch/fx/_symbolic_trace.py:420: UserWarning: Was not able to add assertion to guarantee correct inputs to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  "Was not able to add assertion to guarantee correct inputs to "
/Users/connolly/opt/anaconda3/envs/python3/lib/python3.7/site-packages/transformers/utils/fx.py:838: UserWarning: Could not compute metadata for call_function target <function layer_norm at 0x7f9e79e20ef0>: Could not run 'aten::native_batch_norm' with arguments from the 'Meta' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::native_batch_norm' is only available for these backends: [CPU, Mkl

GraphModule(
  (embeddings): Module(
    (word_embeddings): Module()
    (token_type_embeddings): Module()
    (position_embeddings): Module()
    (LayerNorm): Module()
  )
  (encoder): Module(
    (layer): Module(
      (0): Module(
        (attention): Module(
          (self): Module(
            (query): Module()
            (key): Module()
            (value): Module()
          )
          (output): Module(
            (dense): Module()
            (LayerNorm): Module()
          )
        )
        (intermediate): Module(
          (dense): Module()
        )
        (output): Module(
          (dense): Module()
          (LayerNorm): Module()
        )
      )
    )
  )
  (pooler): Module(
    (dense): Module()
  )
)